In [123]:
/ read data
t: ("DSTFF";enlist ",") 0:`$"./trade.csv";
q: ("DSTFFFF";enlist ",") 0:`$"./quote.csv";
q: select from q where ask>bid, (time within (09:30;11:29:59.999)) or time within (13:00;14:59:59.999)
\c 20 200

In [124]:
/ minutely data of trade
mt: select open:first price, high:max price, low:min price, close:last price, vol:sum size, turnover:sum price * size by sym, date, time.minute from t;
mt: update rtn:-1+close%prev close by sym,date from mt;

In [125]:
/ compute spread and qsize
q: update spread:10000*(ask-bid)%0.5*ask+bid, qsize:0.5*asize+bsize from q;
/ minutely data of quote
mq: select avg spread, avg qsize by date, sym, time.minute from q

In [126]:
/ combine minutely date of trade and quot
m: mt lj mq;
m

sym         date       minute| open    high    low     close   vol     turnover     rtn           spread   qsize   
-----------------------------| ------------------------------------------------------------------------------------
000001.SZSE 2020.12.02 09:25 | 19.93   19.93   19.93   19.93   510912  1.018248e+07                                
000001.SZSE 2020.12.02 09:30 | 19.95   19.98   19.8499 19.86   1468799 2.924391e+07 -0.003512293  12.3238  13624.15
000001.SZSE 2020.12.02 09:31 | 19.86   19.9    19.82   19.84   1036813 2.058599e+07 -0.001007049  11.33195 11594.02
000001.SZSE 2020.12.02 09:32 | 19.84   19.97   19.84   19.93   876541  1.744256e+07 0.00453629    10.78737 17657.92
000001.SZSE 2020.12.02 09:33 | 19.97   20.0299 19.93   20.02   758831  1.517101e+07 0.004515805   10.25897 14381.48
000001.SZSE 2020.12.02 09:34 | 19.98   20.0099 19.95   20      606188  1.210987e+07 -0.000999001  16.4879  14880.85
000001.SZSE 2020.12.02 09:35 | 20      20.0099 19.95   19.95   855245  1

In [127]:
/ daily data 
d: select sum vol, sum turnover, avg spread, avg qsize, last close, vol5:(dev rtn)*sqrt 240 by sym,date from m;
d: update rtn:-1+close%prev close by sym from d;
d

sym         date      | vol          turnover     spread   qsize    close   vol5       rtn         
----------------------| ---------------------------------------------------------------------------
000001.SZSE 2020.12.02| 8.893853e+07 1.758683e+09 7.516636 28644.88 19.63   0.02344508             
000001.SZSE 2020.12.03| 7.144523e+07 1.393157e+09 6.450216 24879.57 19.54   0.0254532  -0.004584819
000001.SZSE 2020.12.04| 8.913478e+07 1.707534e+09 6.432471 36507.19 19.3    0.02668121 -0.0122825  
000001.SZSE 2020.12.07| 8.738066e+07 1.655428e+09 6.567784 24918.27 18.91   0.02211691 -0.02020725 
000001.SZSE 2020.12.08| 7.196475e+07 1.346027e+09 6.641968 29905.66 18.71   0.01993716 -0.01057641 
000001.SZSE 2020.12.09| 6.772212e+07 1.277457e+09 6.468631 28216.97 18.71   0.02121278 0           
000001.SZSE 2020.12.10| 6.078666e+07 1.131031e+09 6.708389 20967.86 18.65   0.02066208 -0.003206841
000001.SZSE 2020.12.11| 8.877939e+07 1.642355e+09 7.086665 28357.11 18.52   0.02333034 -0.006970509


In [128]:
/ a1: ans of Q1
a1:select ADV:avg vol, ADTV:avg turnover, Volatility:dev rtn, Volatility5:avg vol5, Spread:avg spread, Quote_Size:avg qsize by Stock:sym from d;
a1

Stock      | ADV          ADTV         Volatility Volatility5 Spread   Quote_Size
-----------| --------------------------------------------------------------------
000001.SZSE| 8.342043e+07 1.563842e+09 0.0162697  0.02340956  7.016116 31656.78  
600030.SHSE| 9.497093e+07 2.775817e+09 0.0145191  0.01606984  4.180168 35301.06  


In [129]:
save `a1.csv 

`:a1.csv


In [130]:
/ Q2
/ minutely date: add volpct
m: update volpct:vol%sum vol by sym,date from m;
m

sym         date       minute| open    high    low     close   vol     turnover     rtn           spread   qsize    volpct     
-----------------------------| ------------------------------------------------------------------------------------------------
000001.SZSE 2020.12.02 09:25 | 19.93   19.93   19.93   19.93   510912  1.018248e+07                                 0.005744552
000001.SZSE 2020.12.02 09:30 | 19.95   19.98   19.8499 19.86   1468799 2.924391e+07 -0.003512293  12.3238  13624.15 0.01651477 
000001.SZSE 2020.12.02 09:31 | 19.86   19.9    19.82   19.84   1036813 2.058599e+07 -0.001007049  11.33195 11594.02 0.01165764 
000001.SZSE 2020.12.02 09:32 | 19.84   19.97   19.84   19.93   876541  1.744256e+07 0.00453629    10.78737 17657.92 0.009855582
000001.SZSE 2020.12.02 09:33 | 19.97   20.0299 19.93   20.02   758831  1.517101e+07 0.004515805   10.25897 14381.48 0.008532084
000001.SZSE 2020.12.02 09:34 | 19.98   20.0099 19.95   20      606188  1.210987e+07 -0.000999001  16.487

In [131]:
/ m5
m5: select avg spread, avg qsize, sum volpct by sym, date, 5 xbar minute from m;
m5

sym         date       minute| spread   qsize    volpct     
-----------------------------| -----------------------------
000001.SZSE 2020.12.02 09:25 |                   0.005744552
000001.SZSE 2020.12.02 09:30 | 12.238   14427.69 0.05337588 
000001.SZSE 2020.12.02 09:35 | 10.32066 13114.77 0.06996587 
000001.SZSE 2020.12.02 09:40 | 12.30148 13842.22 0.05185609 
000001.SZSE 2020.12.02 09:45 | 9.058464 14133.25 0.03369607 
000001.SZSE 2020.12.02 09:50 | 8.515195 12671.66 0.02730214 
000001.SZSE 2020.12.02 09:55 | 8.754332 18116.86 0.02663499 
000001.SZSE 2020.12.02 10:00 | 11.02709 14998.52 0.02436063 
000001.SZSE 2020.12.02 10:05 | 7.104375 18026.26 0.01767074 
000001.SZSE 2020.12.02 10:10 | 6.829497 25682.93 0.01936488 
000001.SZSE 2020.12.02 10:15 | 6.273198 51132.54 0.01580851 
000001.SZSE 2020.12.02 10:20 | 7.247258 9389.885 0.01244876 
000001.SZSE 2020.12.02 10:25 | 7.387738 10984.83 0.01557659 
000001.SZSE 2020.12.02 10:30 | 6.333285 11678.6  0.01042197 
000001.SZSE 2020.12.02 1

In [132]:
a2: select vol5:(dev rtn)*sqrt 240 by sym, 5 xbar minute from m where sym = `600030.SHSE;
a2

sym         minute| vol5      
------------------| ----------
600030.SHSE 09:25 |           
600030.SHSE 09:30 | 0.02715607
600030.SHSE 09:35 | 0.02750667
600030.SHSE 09:40 | 0.02287371
600030.SHSE 09:45 | 0.02083961
600030.SHSE 09:50 | 0.02102274
600030.SHSE 09:55 | 0.02105505
600030.SHSE 10:00 | 0.0191641 
600030.SHSE 10:05 | 0.01666364
600030.SHSE 10:10 | 0.01742796
600030.SHSE 10:15 | 0.01581928
600030.SHSE 10:20 | 0.01690071
600030.SHSE 10:25 | 0.01514775
600030.SHSE 10:30 | 0.01554634
600030.SHSE 10:35 | 0.01742902
..


In [133]:
a2: a2 lj (select avg spread, avg qsize, avg volpct by sym,minute from m5 where sym = `600030.SHSE);
a2

sym         minute| vol5       spread   qsize    volpct     
------------------| ----------------------------------------
600030.SHSE 09:25 |                              0.005084981
600030.SHSE 09:30 | 0.02715607 5.08496  27922.96 0.04561933 
600030.SHSE 09:35 | 0.02750667 4.363957 27875.45 0.03559443 
600030.SHSE 09:40 | 0.02287371 4.438408 31233.66 0.03642824 
600030.SHSE 09:45 | 0.02083961 4.361236 27797.76 0.03169343 
600030.SHSE 09:50 | 0.02102274 4.293779 34704.57 0.02819256 
600030.SHSE 09:55 | 0.02105505 4.442459 31268.81 0.02758852 
600030.SHSE 10:00 | 0.0191641  4.312367 30726.31 0.02529344 
600030.SHSE 10:05 | 0.01666364 4.261444 31022    0.02246315 
600030.SHSE 10:10 | 0.01742796 4.16108  33049.39 0.02242542 
600030.SHSE 10:15 | 0.01581928 4.152558 28358.27 0.01562317 
600030.SHSE 10:20 | 0.01690071 4.151754 28778.06 0.01717308 
600030.SHSE 10:25 | 0.01514775 4.212934 28100.86 0.01802133 
600030.SHSE 10:30 | 0.01554634 4.103536 28958.2  0.01983326 
600030.SHSE 10:35 | 0.01

In [135]:
save `a2.csv 

`:a2.csv
